# Introduction to AlgoKit Utils Python

This Jupyter notebook is designed to introduce you to the AlgoKit Utils Python library through a series of demonstrations of Algorand concepts applied through code.

The notebook code is designed so that it can be run in its entirety or in individual steps from top to bottom. Each of the sections has an explanation of what the subsequent code will accomplish and a link to the Algorand Developer Portal for additional reading on the topic.

## Initital Imports and Setup

In [ ]:
import logging
from time import sleep

from algokit_utils import (
    AlgoAmount,
    AlgorandClient,
    AssetCreateParams,
    AssetOptInParams,
    AssetTransferParams,
    PaymentParams,
)
from algokit_utils.config import config
from dotenv import load_dotenv

# Configure Utils and Logging
config.configure(populate_app_call_resources=True)

# Set up logging and load environment variables
logging.basicConfig(
    level=logging.DEBUG, format="%(asctime)s %(levelname)-10s: %(message)s"
)
logger = logging.getLogger(__name__)
load_dotenv()

## Initialize an Algorand Client

To get started with AlgoKit Utils, first initialize an Algorand Client that will be used to interact with the chain. For learning and development, use LocalNet so that you can reset the chain as needed and have access to the genesis accounts with all 10B Algo.

The client's methods are your gateway to a wide array of functionality to craft and send transactions, query chain data, and manage accounts. This client is a stateful class that remembers things like the network it is connected to, the signing keys for accounts it has created, and suggested transaction parameters from the node.

You can learn more about the `AlgorandClient` at https://dev.algorand.co/algokit/utils/algokit-clients/#algorand-client-gateway-to-the-blockchain.

In [ ]:
algorand = AlgorandClient.default_localnet()

## Create LocalNet Accounts

Next, use the client to create two LocalNet accounts, Alice and Bob, funded with 100 Algos each. This will enable Alice and Bob to send transactions and create tokens.

Learn more about LocalNet's capabilties to accelerate your development and testing efforts at https://dev.algorand.co/algokit/algokit-cli/localnet/.

In [ ]:
alice = algorand .account.from_environment("ALICE", AlgoAmount(algo=100))
bob = algorand.account.from_environment("BOB", AlgoAmount(algo=100))
print(
    f"\nAlice's generated account address: {alice.address}. \nView her account on Lora at https://lora.algokit.io/localnet/account/{alice.address}."
)
print(
    f"\nBob's generated account address: {bob.address}. \nView his account on Lora at https://lora.algokit.io/localnet/account/{bob.address}."
)

## Send Payment Transaction

Now, let's have Alice send an Algo payment to Bob to see a very simple type of transaction. See https://dev.algorand.co/concepts/transactions/types/#payment-transaction for more information.

In [ ]:
pay_result = algorand.send.payment(
    PaymentParams(
        sender=alice.address,
        receiver=bob.address,
        amount=AlgoAmount(
            algo=2
        ),  # The AlgoAmount class is a helper to be explicit about amounts between microAlgos and Algos
        note=b"Hi, Bob!",
    )
)
print(
    f"\nPay transaction confirmed with TxnID: {pay_result.tx_id}. \nView it on Lora at https://lora.algokit.io/localnet/transaction/{pay_result.tx_id}."
)

## Create an Algorand Standard Asset (ASA)

Now, let's have Alice create a token using an Algorand Standard Asset, which is often called an ASA or just asset. Note that some asset parameters are mutable, meaning they can be modified after creation, while others are immutable and can never be changed. See the docs to learn which parameters are (im)mutable: https://dev.algorand.co/concepts/assets/overview/.

In [ ]:
create_asset_result = algorand.send.asset_create(
    AssetCreateParams(
        sender=alice.address,
        asset_name="My First ASA",  # A human-readable name for the asset
        unit_name="MFA",  # A short ticker; this is not a unique identifier
        total=1_000_000_000_000,  # The true supply of indivisible units
        decimals=6,  # Used for displaying the asset amount off chain
        default_frozen=False,  # This asset can be transferred freely
        manager=alice.address,  # Account that can change the asset's config
        reserve=alice.address,  # Account to hold non-circulating supply
        freeze=alice.address,  # Account that can freeze asset holdings
        clawback=alice.address,  # Account that can revoke asset holdings
        url="https://algorand.co/algokit",  # Often used to point to metadata
        note=b"This is my first Algorand Standard Asset!",
    )
)

# Store the Asset ID Alice created in a variable for later use in the script
# This UInt64 Asset ID is a unique identifier for the asset on the chain
created_asset = create_asset_result.asset_id
print(
    f"\nAsset ID {created_asset} create transaction confirmed with TxnID: {create_asset_result.tx_id}."
)
print(
    f"\nView it on Lora at https://lora.algokit.io/localnet/asset/{created_asset}."
)

## Get ASA Information

Here we'll get ASA information from algod's /v2/assets REST API endpoint. The response will contain all of the asset's current parameters from the ledger on chain.

You can use interactive API tools like Scalar, Swagger, and Postman to explore Algorand's algod REST API endpoints here: https://dev.algorand.co/reference/rest-api/overview/#algod-rest-endpoints.

In [ ]:
asset_info = algorand.asset.get_by_id(created_asset)
print(
    f"\nAsset information from algod's /v2/assets/{{asset-id}} REST API endpoint: {asset_info}."
)

## Opt into an ASA

Next, have Bob opt in to the ASA so that he will be able to hold it when Alice sends it to him. On Algorand, an account must always opt into the ASA before anyone can send that asset to it. Details about opting in and out of assets are available here: https://dev.algorand.co/concepts/assets/asset-operations/#opting-in-and-out-of-assets.

In [ ]:
bob_opt_in_result = algorand.send.asset_opt_in(
    AssetOptInParams(
        sender=bob.address,
        asset_id=created_asset,
    )
)
print(
    f"\nAsset opt-in transaction confirmed with TxnID: {bob_opt_in_result.tx_id}. \nView it on Lora at https://lora.algokit.io/localnet/transaction/{bob_opt_in_result.tx_id}."
)

## Send an Asset Transfer

Now Alice can send some of the ASA to Bob, who is now able to receive it. Asset transfer transaction docs are available here: https://dev.algorand.co/concepts/transactions/types/#asset-transfer-transaction/.

In [ ]:
send_asset_result = algorand.send.asset_transfer(
    AssetTransferParams(
        sender=alice.address,
        receiver=bob.address,
        asset_id=created_asset,
        amount=3_000_000,  # The amount is in the smallest unit of the asset
        note=b"Have a few of my first ASA!",
    )
)
print(
    f"\nAsset transfer transaction confirmed with TxnID: {send_asset_result.tx_id}. \nView it on Lora at https://lora.algokit.io/localnet/transaction/{send_asset_result.tx_id}."
)

## Get Account Information

Here we'll get the current ledger state for Bob's account, including Algo balance, asset balances with some asset information, as well as application-related information like local state, and more. You can learn more about Algorand accounts at https://dev.algorand.co/concepts/accounts/overview/.

In [ ]:
bob_account_info = algorand.account.get_information(bob.address)
print(
    f"\nBob's account information from algod's /v2/accounts/{{address}} REST API endpoint: \n{bob_account_info}."
)

## Build an Atomic Transaction Group

Now we'll build an atomic transaction group with two transactions which, once grouped, must succeed together or fail together. Utils provides this fluent way of chaining method calls to build the group rather than using an SDK to create transactions and manually group them.

When submitted, these transactions will be either confirmed or rejected together; one cannot go through without the other under any circumstance. Atomic transaction group docs can be found at https://dev.algorand.co/concepts/transactions/atomic-txn-groups/.

In [ ]:
group_result = (
    algorand.send.new_group()
    .add_payment(
        PaymentParams(
            sender=bob.address,
            receiver=alice.address,
            amount=AlgoAmount(algo=1),
            note=b"Thanks, Alice!",
        )
    )
    .add_asset_transfer(
        AssetTransferParams(
            sender=bob.address,
            receiver=alice.address,
            asset_id=created_asset,
            amount=1_000_000,
            note=b"Sending back one of your token!",
        )
    )
).send()
print(
    f"\nAtomic transaction group confirmed with first TxnID: {group_result.tx_ids[0]}. \nView it on Lora at https://lora.algokit.io/localnet/transaction/{group_result.tx_ids[0]}."
)

## Get Transaction Information from an Indexer

Let's use another Algorand API, the indexer, to do a historical transaction search. You can explore the Indexer's capabilities interactively through Scalar, Swagger, or Postman here: https://dev.algorand.co/reference/rest-api/overview/#indexer-rest-endpoints.

While the indexer is a powerful PostgreSQL database, indexing the entire chain's activity requires significant hardware resources, so developers should be cautious about building applications that rely on the indexer APIs or explore alternative infrastructure tools such as Conduit, which can be configured to index only relevant transactions that meet certain criteria. Conduit docs can be found at https://dev.algorand.co/nodes/installation/conduit-installation/. 

In [ ]:
# We add a short delay here because indexer can be a bit slow on LocalNet.
# Engineering will be working on improving this in the future.
print(
    "\nSleeping for 30 seconds to let the LocalNet indexer to catch up, which can sometimes take a moment."
)
sleep(30)

# Here the AlgorandClient exposes the underlying SDK indexer client to build
# the query with various parameters. Be mindful of how broad the query is
# to avoid long-running requests or needing to page through many results.
transfer_search_results = algorand.client.indexer.search_transactions(
    asset_id=created_asset,
    txn_type="axfer",
)
found_txn_ids = [txn["id"] for txn in transfer_search_results["transactions"]]
print(
    f"\nAsset transfer transaction IDs found by searching the indexer: {found_txn_ids}."
)